In [33]:
import scanpy as sc
import numpy as np
#import squidpy as sq
import anndata2ri
import logging
from scipy.sparse import issparse
from CSCORE.CSCORE_IRLS import CSCORE_IRLS
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
from scipy.sparse import issparse
import pandas as pd

In [5]:
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

/scratch/3285911.1.academic-gpu/ipykernel_1084831/1068135002.py:3: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [6]:
%%R
library(SPARK)
library(Seurat)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
[1] "value"


In [34]:
#adata = sq.read.visium("/projectnb/cs598/projects/singleCell/data/filtered_feature_bc_matrix/")
# Load the 10x Genomics feature-barcode matrix
#adata = sc.read_10x_mtx(
#    "/projectnb/cs598/projects/singleCell/data/filtered_feature_bc_matrix/",  # Path to extracted folder
#    var_names="gene_symbols",  # Use gene symbols instead of gene IDs
#    cache=True
#)
#
# Save as an H5AD file
#adata.write("spatial_heart_visium_processed.h5ad")
#
#print("Saved as spatial_heart_visium_processed.h5ad")

NameError: name 'squidpy' is not defined

In [7]:
adata = sc.read_h5ad("/projectnb/cs598/projects/singleCell/data/visium-OCT_AVN_raw.h5ad")

In [21]:
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=3000)
hvg_list = adata.var[adata.var['highly_variable']].index.tolist()

adata = adata[:, hvg_list]

# Select 5,000 random cells
random_cell_indices = np.random.choice(adata.n_obs, 5000, replace=False)
adata = adata[random_cell_indices, :]

print(adata.shape)

/usr3/graduate/aleshchk/.local/lib/python3.12/site-packages/scanpy/preprocessing/_highly_variable_genes.py:172: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns["hvg"] = {"flavor": flavor}


(5000, 3000)


In [ ]:
if issparse(adata.X):
    if not adata.X.has_sorted_indices:
        adata.X.sort_indices()
ro.globalenv["adata"] = adata

In [ ]:
%%R
seurat_obj = as.Seurat(adata, counts="X", data = NULL)
seurat_obj = RenameAssays(seurat_obj, originalexp = "RNA")

In [10]:
%%R
rawcount = seurat_obj@assays$RNA@counts
info = data.frame(adata@int_colData$reducedDims$spatial)
colnames(rawcount) <- rownames(info)

         X1    X2
  3694  1483
  6747  2087
 12510  4279
 15871 12171
  5016  9760
  4001  8771
  9121  2689
  2285 12900
  4485  1684
 14652  6061
 13706 15510
  7307  3468
  3555  7194
  8879  6232
  4805  6215
  3548  8769
 15976 13944
 11831  4276
 14872  7637
  9652 10765
 14428  5666
  4585  4638
  9681  4070
  7627  7999
  1860  6596
  8418  8200
  2308  7779
 11604  4275
 12499  6642
  4029  2469
  6612  7010
  4687  7199
  9232  3280
  5726  2674
  5366  7202
 13064  6842
  3804  2074
  4568  8577
  5266  4247
  6284  4449
  6692 14691
  9799  3085
  9785  6236
 15207  8623
 12282  4672
  7868  4456
 10464  6239
 13636  5466
  4366  3062
  7824 14696
  6522  1692
  6389  6222
 14157 15906
 11269  3289
  2996  5813
 16192 16309
  7312  2287
  6830  8981
  3237  2269
  4929  3655
 12378  8414
 16213 11582
  9339  4659
  9335  5447
  3904  5029
  4914  7200
  4134  4242
  2648  7584
  7424  2484
  4701  4048
  4934  2473
  9548  8599
  4915  6806
  5131  9170
 12047  6640
 15978 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  4819  8195
  4375  6224
  8192 13328
  4046  3663
  5862  2490
 10465 10975
  3462  8189
  5151  9968
  7753 10176
  9372  1914
  8878 11755
  6720 13518
  1638 11331
  4106 16460
  4045  4057
  4831  5438
  1732 15860
  4785 16463
  3351  7598
  5931 12924
  4593  8194
  7672  2497
  9888 13532
 13782  2523
  5402  4063
  3673 11734
  3580  6811
  2546 10548
  8774  9589
  5617  6820
  6765  2887
  3921  6616
  9561 10577
  8343  4469
 10154  4083
 10386  2902
  7758  8994
  4675 15478
  9441 12348
  7394 14702
  3457  9370
  5919 15680
  2449  6807
  6844 10959
  6843 11353
 10391  1721
  6935 16275
  2999 10549
  8883 10574
  6613 12140
  7206  5645
  3348  8385
  4499  3665
  6370 16076
  9330 11757
  6638  6234
  6750  6431
  8304 13525
  2650 12714
  6948 13125
 12875  2913
  2302 14878
  4380  5043
  5820 12333
  8752 14708
 11400  3891
  4937  7211
  8517 16676
 14575  2329
  4225 15083
  5282  5834
  6834 13322
  6640  5840
  7847 14704
  3204 15669
  2533 13697
  9446 11167

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  9822 10588
  9371 10193
  7915  6843
  4855  8601
 13230  6667
  6119  3687
 10850  8035
  5435  4865
  3508  6235
  7931  2908
  6117  4081
  5182 11553
  9593 11374
  7366  2709
  9488  9210
  5850 14310
 11756  7645
 10837 11183
 10374 13935
  6076 14311
  3063  4265
  3974  2695
 12642 12371
  5546  5456
  3410  2496
  4159 12926
  7823  1530
  9144 10586
  4164 11746
  4991  2896
 12090  9024
  4642  5452
  5976 10966
  4087  2892
 15245 11791
  3708 12531
  4986  4076
 10393  9214
 15268  5889
  6802  2510
  8608  3304
  1777 14687
  5205  6045
  5312  7422
  2164  3081
  3164  7217
  3065  3872
  8592  7239
  8707  6649
  9264  8815
  3615  7612
  9945  8031
  1910  9769
 10943 12954
  2597  7805
 11625 12170
  7575  7038
  3512  5054
 11965 11974
  2353 12132
  2344 14493
  2486  7214
  2379  5837
  3585 15088
 15357 11988
  6534 12739
 10167  9213
  5763  7818
  2138  9377
  8691 10584
  2706  8789
 15488  7463
 10274 10590
  3158  8791
 11854 11384
  3141 13119
  6656 10576

...

  4645  9471
  7626  9477
 12193 11215
  6537  7227
 10025  2221
  4350  7914
  9617  7406
  9024  5676
 12905  3091
 13998  2920
  8718  9998
 12806  2918
 11806  6028
 12299  7585
  6732  9302
  3166  4108
  7824  9477
  4556  4457
  9210 11900
  7918 12416
  8715 11035
  4450  7396
  7542  2043
  7043  2906
 13002  3956
  7920 11379
 13789  7761
  6423 14315
  9317  8270
  3748 11025
  9721  5159
  2663  7046
  9213 10863
 10805  9483
  7819 12243
 11993 11560
  7635  4982
  9907 11383
  2564  6527
 10712  6198
  6041  7053
  6341  5844
 10812  6026
  8238  1525
 11297 12077
 12883 13810
  2966  4454
  4845  8780
 11210  5681
  8628  4984
  7930  6193
  8236  2563
  7230  8439
 11702  7929
  8317 11380
  8915 10690
 10898 12768
  1664  9810
 12290 12080
  2652 12233
 12001  7757
  2158 11021
 12497  7931
  9430  1874
 10702 11385
 10219  4641
  7630  7402
  7021 13625
  8312 14146
 11207  7409
 13402  2919
  5540  8954
  9214 10172
  7440  3253
  4755  4457
 13583 11564
  7035  6709

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  5973  6843
 11434  8756
 13924  5476
 11940  4089
  2793  6837
 13215 12217
  5370  9954
 11148  2705
  9746  7888
 10733 11002
  6766  7536
 12322 11697
  7361  8229
  8762  3391
 10350  4086
  9047  9788
  2991  7529
 11726 11696
  5977  4769
  6469  7363
  4784  5112
  3097  3898
 10138 10655
 12721 11006
 14014 10144
  6760 10648
 11537  6855
  6662 10129
  7357 10304
  8657  5984
 14007 13602
  6461 10820
  8564  3045
 11822 12906
  6273  5979
 12434  5300
  9451  6332
 11629 10139
  9042 12209
 10153  3394
  9354  5467
 13910 12392
 11145  4088
  5965 10647
  7562  7192
 10644  6334
  5879  3904
  4482  7013
 10152  3740
 14317  7552
  7657  9094
 10134 12730
  5970  8226
  8947  9961
 14126  3748
 11740  4780
  4188  5111
 11839  4953
 10744  5816
 13631  2883
 11539  5817
  8254  8750
  9949  5814
  5382  4076
 10834 10484
  6768  6499
  4285  5975
 11038  7718
  8845 11517
  5184  3730
  4778  7878
  7957  8230
  6468  7709
 12926  7895
 10331 13422
  3888  5975
  5865 10819

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  4511  5271
  1430  8312
 11754  8894
  7841  7589
 10006  5562
  3928  7154
  7759  4837
 11838  7301
  7591  9473
  7591  8893
  8175  5851
  7508  7010
  8174  8169
  7341  9038
 10505  7300
  7174  9907
 10922  6287
  2263  5415
  4011  6429
 11254 11212
  6842  7879
  5510  7299
 10423  5417
  2429  8892
 10172  8749
 10839  4403
  5926  8024
  2763  4256
  1265  4255
  4095  5705
 10006  5852
  5677  3532
 11171  9619
  5677  6430
 10339  8749
 10754 10343
 10422  7445
 10256  5707
  1515  1793
  8673  8459
 11172  4114
  5511  3243
  4594  7154
  7758  8604
  5261  4546
 11754 10053
 11172  7011
 11171  8749
 10923  3969
  9258  3389
  8591  5706
  6508  9038
  5178  3243
 11337 10198
  6592  8024
  1931  4256
 10338  9908
  3512  6719
  5344  4691
  7258  6865
  7924  7155
  3929  2808
  6675  8458
  8839 10197
  6176  8748
  1513  8747
  4261  8892
  2264  3676
 11088  6866
  8591  5127
  9341  3533
 11256  4548
  3679  4401
 11837 10198
  4179  4111
  2596  5125
  7092  6575

In [11]:
%%R 
location <- as.matrix(info)
mt_idx <- grep("mt-",rownames(rawcount))
if(length(mt_idx)!=0){
    rawcount    <- rawcount[-mt_idx,]
}

[1] "value"


In [27]:
adata.obs['n_counts'] = np.array(np.sum(adata.X, axis = 1))

if issparse(adata.X):
    if not adata.X.has_sorted_indices:
        adata.X.sort_indices()
ro.globalenv["adata"] = adata

In [28]:
adata.shape

(5000, 3000)

In [29]:
%%R
seurat_obj = as.Seurat(adata, counts="X", data = NULL)
seurat_obj = RenameAssays(seurat_obj, originalexp = "RNA")
res = SCTransform(object=seurat_obj, vst.flavor = "v2", variable.features.n = 1000 , method = "glmGamPoi", verbose = FALSE)

[1] "value"


In [38]:
var_cand = list(ro.r("seurat_obj@assays$RNA@counts@Dimnames[[1]]"))
adata.var_names = var_cand
gene_list = list(ro.r("rownames(res@assays$SCT@scale.data)"))
norm_x = ro.r("res@assays$SCT@scale.data")
exp_matrix = pd.DataFrame(norm_x, index=gene_list)


adata_new = adata[:,gene_list]

df = pd.DataFrame(adata_new.X.todense(), index=adata_new.obs_names, columns = adata_new.var_names)

df.to_csv("visium_spatial_data.csv")

counts = adata_new.X
seq_depth = adata_new.obs['n_counts'].values
counts = counts.toarray()
B_cell_result = CSCORE_IRLS(np.array(counts), seq_depth)
p_value = B_cell_result[1]
cor_matrix = (p_value<0.005)*1
print(cor_matrix)

exp_matrix.to_csv("/projectnb/cs598/projects/singleCell/data/heart_spatial_expression.csv")
cor_matrix = pd.DataFrame(cor_matrix, index = gene_list, columns = gene_list)
cor_matrix.to_csv("/projectnb/cs598/projects/singleCell/data/heart_spatial_pvalue.csv")

IRLS converged after 3 iterations.
0.0094% co-expression estimates were greater than 1 and were set to 1.
0.0000% co-expression estimates were greater than 1 and were set to 1.
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 1 ... 1 1 1]
 ...
 [0 0 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]]
